In [2]:
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from gym import ActionWrapper
from gym.wrappers.monitoring.video_recorder import VideoRecorder


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/kpradjinata/opt/anaconda3/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/kpradjinata/opt/anaconda3/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/Users/kpradjinata/Documents/Research/OpenAIGym/venv/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/kpradjinata/Documents/Research/OpenAIGym/venv/lib/python3.9/site-packages/traitlets/con

In [3]:
class PolicyNetwork(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(PolicyNetwork, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(state_dim, 64),  # state_dim should be 4
            nn.ReLU(),
            nn.Linear(64, action_dim)  # action_dim should be 2
        )

    def forward(self, x):
        return self.fc(x)

In [4]:
env = gym.make('CartPole-v1')
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

policy = PolicyNetwork(state_dim, action_dim)
optimizer = optim.Adam(policy.parameters(), lr=0.01)


state, _ = env.reset()  # Use this instead of just env.reset()
state, _ = env.reset()
print("Initial state:", state)

action_probs = policy(torch.FloatTensor(state))

Initial state: [ 0.0015697  -0.03605074  0.01477886  0.02003803]


In [5]:
state, _ = env.reset()
state = state['observation'] if isinstance(state, dict) else state
print("State shape:", state.shape)
print("PolicyNetwork input shape:", policy.fc[0].in_features)
action_probs = policy(state if isinstance(state, torch.Tensor) else torch.FloatTensor(state))

State shape: (4,)
PolicyNetwork input shape: 4


In [6]:
import torch
import torch.nn.functional as F
from torch.distributions import Categorical

# Your original logits
logits = torch.tensor([[1.3031, -0.3031]])

# Create the Categorical distribution with logits
dist = Categorical(logits=logits)

# You can now use the distribution for sampling or other operations
action = dist.sample()
log_prob = dist.log_prob(action)

In [11]:
from tqdm import tqdm

num_episodes = 1000
max_steps = 500
rewards = []

# Add a progress bar for episodes
for episode in tqdm(range(num_episodes), desc="Training Progress"):
    state, _ = env.reset()
    episode_reward = 0
    log_probs = []
    
    for step in range(max_steps):
        state_tensor = torch.FloatTensor(state).unsqueeze(0)
        action_probs = torch.nn.functional.softmax(policy(state_tensor), dim=-1)
        distribution = torch.distributions.Categorical(action_probs)
        action = distribution.sample()
        log_prob = distribution.log_prob(action)
        
        # Use the wrapper function to avoid np.bool8 error
        next_state, reward, done, truncated, _ = step_with_bool_fix(env, action.item())
        episode_reward += reward
        log_probs.append(log_prob)
        
        if done or truncated:
            break
        
        state = next_state
    
    rewards.append(episode_reward)
    
    # Update policy only if we have collected any log probabilities
    if log_probs:
        loss = -torch.stack(log_probs).sum() * episode_reward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if episode % 10 == 0:
        avg_reward = np.mean(rewards[-10:])
        print(f"Episode {episode}, Average Reward: {avg_reward:.2f}")

env.close()


Training Progress:   0%|                       | 5/1000 [00:00<00:47, 20.96it/s]

Episode 0, Average Reward: 121.00


Training Progress:   1%|▎                     | 14/1000 [00:00<00:44, 22.27it/s]

Episode 10, Average Reward: 113.70


Training Progress:   2%|▌                     | 23/1000 [00:01<00:47, 20.75it/s]

...

Training Progress:  98%|████████████████████▋| 984/1000 [01:24<00:00, 19.62it/s]

Episode 980, Average Reward: 118.00


Training Progress:  99%|████████████████████▊| 994/1000 [01:24<00:00, 24.19it/s]

Episode 990, Average Reward: 106.00


Training Progress: 100%|████████████████████| 1000/1000 [01:25<00:00, 11.74it/s]
